In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [74]:
def crawl_query(query):
    req = requests.get(f"https://www.bing.com/search?q={query}" + "&answerCount=5&promote=webpages%2Cvideos", headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'})
    result_str = '<html><table style="border: none;">'  # Initializing the HTML code for displaying search results
    count_str = ''
    if req.status_code == 200:  # Status code 200 indicates a successful request
        bs = BeautifulSoup(req.content, features="html.parser")  # converting the content/text returned by request to a BeautifulSoup object
        search_result = bs.find_all("li",class_= "b_algo")  # 'b_algo' is the class of the list object which represents a single result
        search_result = [str(i).replace("<strong>", "") for i in search_result]  # removing the <strong> tag
        search_result = [str(i).replace("</strong>", "") for i in search_result]  # removing the </strong> tag
        result_list = []

        for n, i in enumerate(search_result):  # iterating through the search results
            individual_search_result = BeautifulSoup(i, features="html.parser")  # converting individual search result into a BeautifulSoup object
            h2 = individual_search_result.find('h2')  # Finding the title of the individual search result
            href = h2.find('a').get('href')  # title's URL of the individual search result
            cite = f'{href[:50]}...' if len(href) >= 50 else href  # cite with first 20 chars of the URL
            url_txt = h2.find('a').text  # title's text of the individual search result
            # In a few cases few individual search results doesn't have a description. In such cases the description would be blank
            description = "" if individual_search_result.find('p') is None else individual_search_result.find('p').text[3:]
            # Appending the result data frame after processing each individual search result
            result_list.append({"Title": url_txt, "URL": href, "Description": description})
            ########################################################
            ######### HTML code to display search results ##########
            ########################################################
            description = description[:200] + '...'
            result_str += f'<tr style="border: none;"><h6><a href="{href}" target="_blank">{url_txt}</a></h6></tr><tr style="border: none;"><h7>{description}</h7></tr><tr style="border: none;"><h6>{""}</h6></tr>'
            if n > 10:
                break

        result_str += '</table></html>'
        count_str = f'<b style="font-size:12px;">Search returned {len(result_list)} results</b>'
        result_df = pd.DataFrame(result_list)

    else:
        result_df = pd.DataFrame({"Title": "", "URL": "", "Description": ""}, index=[0])
        result_str = '<html></html>'
        count_str = '<b style="font-size:20px;">Looks like an error!!</b>'

    return result_df, result_str, count_str

In [75]:
#st.markdown('<h1 style="background-color: gainsboro; padding-left: 10px; padding-bottom: 20px;">Search Engine Scraper</h1>', unsafe_allow_html=True)
# query = st.text_input('', help='Enter the search string and hit Enter/Return')
query='réparation grille pain SEB comment faire'
query = query.replace(" ", "+") #replacing the spaces in query result with +

In [76]:
df,result_str,count_str=crawl_query(query)

In [77]:
df

,Title,URL,Description
0,"Mon grille pain ne se bloque plus, que faire -...",https://www.youtube.com/watch?v=HhvMarglKjs,28 sept. 2020 · Nous vous montrons dans ce tut...
1,Comment réparer mon Grille Pain moi-même ? Gui...,https://www.spareka.fr/comment-reparer/electro...,Un fusible ou un fil coupé peuvent entraîner d...
2,Comment reparer un grille pain seb,http://www.comment-reparer.fr/comment-reparer-...,"En vue d'une bonne réparation, Barnabé a listé..."
3,Réparer . Grille-pain Seb - CommentReparer.com,https://www.commentreparer.com/reparations/55-...,6 sept. 2016 · Demander une réparation Pas tro...


In [67]:
response = requests.get("https://example.com/")
print(response.request.headers)

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
